In [8]:
import sys
import os 

import functions
from functions import *

# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
bounds = unpickle_data("../data/interim/bounds.pickle")
myorigins = get_origins(
    places = bounds,
    population_file = "../data/interim/pop_phl_national_capital_region.tif",
    window_size = 50,
    use_pickle = False,
    do_pickle_result = True
)

#myorigins = origins_snap_osm(myorigins, "./data/networks/ncr_unclean_1.pickle")
myorigins.head()

Unpickled data at ../data/interim/bounds.pickle: 
 Rows: (2, 8);
119 regions need splitting
done.44
We now have 600 regions of min size 50, 423 will be split in next round
We have 423 regions of size 50, 423 with population >0
All origins:(423, 17); Relevant Origins:(263, 17)
Pickled origins


,window,left_x,right_x,top_y,bottom_y,left_lon,top_lat,right_lon,bottom_lat,center_lon,center_lat,count,nancount,tot_pop,split,geometry,geo_grid
0,50,0,50,300,350,120.906389,14.701944,120.920278,14.688056,120.913333,14.695000,2500,1,1101.379570,True,POINT (120.91333 14.69500),"POLYGON ((120.9063888888921 14.68805555555067, 120.9063888888921 14.70194444443957, 120.920277777781 14.70194444443957, 120.920277777781 14.68805555555067, 120.9063888888921 14.68805555555067))"
1,50,50,100,300,350,120.920278,14.701944,120.934167,14.688056,120.927222,14.695000,2500,56,2689.352852,True,POINT (120.92722 14.69500),"POLYGON ((120.920277777781 14.68805555555067, 120.920277777781 14.70194444443957, 120.9341666666699 14.70194444443957, 120.9341666666699 14.68805555555067, 120.920277777781 14.68805555555067))"
2,50,50,100,350,400,120.920278,14.688056,120.934167,14.674167,120.927222,14.681111,2500,48,13662.802923,True,POINT (120.92722 14.68111),"POLYGON ((120.920277777781 14.67416666666177, 120.920277777781 14.68805555555067, 120.9341666666699 14.68805555555067, 120.9341666666699 14.67416666666177, 120.920277777781 14.67416666666177))"
3,50,100,150,150,200,120.934167,14.743611,120.948056,14.729722,120.941111,14.736667,2500,98,3719.568907,True,POINT (120.94111 14.73667),"POLYGON ((120.9341666666699 14.72972222221737, 120.9341666666699 14.74361111110627, 120.9480555555588 14.74361111110627, 120.9480555555588 14.72972222221737, 120.9341666666699 14.72972222221737))"
4,50,100,150,200,250,120.934167,14.729722,120.948056,14.715833,120.941111,14.722778,2500,400,12661.796216,True,POINT (120.94111 14.72278),"POLYGON ((120.9341666666699 14.71583333332847, 120.9341666666699 14.72972222221737, 120.9480555555588 14.72972222221737, 120.9480555555588 14.71583333332847, 120.9341666666699 14.71583333332847))"
